In [14]:
from configs import MODEL_PATH
import os
import numpy as np
from keras.datasets import mnist
from keras.saving.saving_api import load_model
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

MODEL_PATH

'D:\\code-example\\my_research\\mnist_cnn\\model'

In [2]:
#load model h5
model = load_model(os.path.join(MODEL_PATH, 'v1_cnn_mnist.h5'))

In [7]:
(_, _), (X_test, Y_test) = mnist.load_data()

In [9]:
#preprocessing
x_test = X_test / 255.0
x_test = x_test.reshape(-1, 28, 28, 1)

y_test = to_categorical(Y_test)

# Normalization

In [21]:
def standardize(x):
    mean = np.mean(x_test)
    std = np.std(x_test)
    return (x - mean) / std

# Evaluate using x_test

In [10]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 27s 69ms/step - loss: 0.0231 - accuracy: 0.9947


[0.023077383637428284, 0.994700014591217]

# Evaluate using generator

In [13]:

# With data augmentation to prevent overfitting
batch_size = 128

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.01,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# datagen.fit(X_train)
test_gen = datagen.flow(x_test, y_test, batch_size=batch_size)
model.evaluate(test_gen)

79/79 [==============================] - 22s 261ms/step - loss: 0.0270 - accuracy: 0.9950


[0.027008920907974243, 0.9950000047683716]

# Make predict using x_test

In [15]:
y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_true_class = np.argmax(y_test, axis=1)

print(np.sum(y_pred_class == y_true_class) / len(y_true_class))

313/313 [==============================] - 25s 75ms/step
0.9947


# Make predict using generator

In [18]:
y_pred = model.predict_generator(datagen.flow(x_test, None, batch_size=batch_size))
y_pred_class = np.argmax(y_pred, axis=1)
y_true_class = np.argmax(y_test, axis=1)

print(np.sum(y_pred_class == y_true_class) / len(y_true_class))

C:\Users\iamca\AppData\Local\Temp\ipykernel_12348\2216938917.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  y_pred = model.predict_generator(datagen.flow(x_test, None, batch_size=batch_size))


0.0968
